In [1]:
import pandas as pd

sprzedaz_df = pd.read_csv("raport_kwartalny_sprzedaz.csv", encoding="cp1250")
katalog_df = pd.read_csv("katalog_produktow.csv", encoding="cp1250")

In [2]:
sprzedaz_df.head(5)

,Data_Transakcji,ID_Produktu_Sprzedaz,Region,Ilosc_Sprzedana,Cena_Jednostkowa_Transakcji,Wartosc_Sprzedazy
0,2025-05-08,1074,PóBnoc,1,44.30,44.30
1,2025-05-13,1068,PóBnoc,5,126.45,632.25
2,2025-04-12,1090,Wschód,2,3974.94,7949.88
3,2025-05-27,1031,Zachód,1,230.14,230.14
4,2025-06-23,1026,Wschód,5,167.64,838.20


In [3]:
sprzedaz_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7500 entries, 0 to 7499
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Data_Transakcji              7500 non-null   object 
 1   ID_Produktu_Sprzedaz         7500 non-null   int64  
 2   Region                       7500 non-null   object 
 3   Ilosc_Sprzedana              7500 non-null   int64  
 4   Cena_Jednostkowa_Transakcji  7500 non-null   float64
 5   Wartosc_Sprzedazy            7500 non-null   float64
dtypes: float64(2), int64(2), object(2)
memory usage: 351.7+ KB


In [4]:
katalog_df.head(5)

,ID_Produktu_Katalog,Nazwa_Produktu,Cena_Katalogowa,Kategoria_Produktu
0,1001,Mie na imi Ósmy,339.36,Akcesoria
1,1002,MaBo Zapalenie,153.68,Biuro
2,1003,Kalendarz GBos,291.73,Oprogramowanie
3,1004,Dzielnica I[,307.01,Akcesoria
4,1005,Lot Centralny,977.54,Elektronika


In [5]:
katalog_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID_Produktu_Katalog  101 non-null    int64  
 1   Nazwa_Produktu       101 non-null    object 
 2   Cena_Katalogowa      101 non-null    float64
 3   Kategoria_Produktu   101 non-null    object 
dtypes: float64(1), int64(1), object(2)
memory usage: 3.3+ KB


### Zad 1

In [7]:
merged_df = sprzedaz_df.merge(katalog_df, how="left",
                              left_on="ID_Produktu_Sprzedaz",
                              right_on="ID_Produktu_Katalog",
                              validate="many_to_one")

brakujace_transakcje = merged_df[merged_df['ID_Produktu_Katalog'].isna()]

liczba_unikalnych_transakcji = brakujace_transakcje['ID_Produktu_Sprzedaz'].nunique()

print(liczba_unikalnych_transakcji)

146


### Zad 2

In [9]:
merged_df = sprzedaz_df.merge(katalog_df,
                             how='inner',
                             left_on='ID_Produktu_Sprzedaz',
                             right_on='ID_Produktu_Katalog')

wartosc_sprzedazy = merged_df['Wartosc_Sprzedazy'].sum()

print(round(wartosc_sprzedazy, 2))

31336539.21


### Zad 3a

In [11]:
merged_df = sprzedaz_df.merge(
    katalog_df,
    how='inner',
    left_on='ID_Produktu_Sprzedaz',
    right_on='ID_Produktu_Katalog',
    validate='many_to_one'
)

avg_prices = merged_df.groupby('Nazwa_Produktu')['Cena_Jednostkowa_Transakcji'].mean().reset_index()

avg_prices = avg_prices.merge(katalog_df[['Nazwa_Produktu', 'Cena_Katalogowa']], how='left', on='Nazwa_Produktu')

avg_prices = avg_prices[avg_prices['Cena_Katalogowa'] != 0]

avg_prices['roznicaprocentowa'] = (
    abs(avg_prices['Cena_Jednostkowa_Transakcji'] - avg_prices['Cena_Katalogowa']) /
    avg_prices['Cena_Katalogowa']
)

max_diff_product = avg_prices.loc[avg_prices['roznicaprocentowa'].idxmax()]

print(max_diff_product['Nazwa_Produktu'])

Miara Przyczyna


### Zad 3b

In [13]:
merged_df = sprzedaz_df.merge(katalog_df,
                             how='inner',
                             left_on='ID_Produktu_Sprzedaz',
                             right_on='ID_Produktu_Katalog')

avg_prices = merged_df.groupby('Nazwa_Produktu')['Cena_Jednostkowa_Transakcji'].mean().reset_index()
avg_prices = avg_prices.merge(katalog_df, how='left', on='Nazwa_Produktu')

avg_prices['roznicaprocentowa_abs'] = (
    abs(avg_prices['Cena_Jednostkowa_Transakcji'] - avg_prices['Cena_Katalogowa']) /
    avg_prices['Cena_Katalogowa']
)

max_diff_product = avg_prices.loc[avg_prices['roznicaprocentowa_abs'].idxmax()]

roznicaprocentowa = (
    (max_diff_product['Cena_Jednostkowa_Transakcji'] - max_diff_product['Cena_Katalogowa']) /
    max_diff_product['Cena_Katalogowa']
) * 100

print(f"{roznicaprocentowa:+.2f}%")

-30.00%


### Zad 4

In [15]:
sprzedaz_df['Wartosc_Obliczona'] = sprzedaz_df['Ilosc_Sprzedana'] * sprzedaz_df['Cena_Jednostkowa_Transakcji']

tolerancja = 0.01

niezgodnosci_df = sprzedaz_df[abs(sprzedaz_df['Wartosc_Sprzedazy'] - sprzedaz_df['Wartosc_Obliczona']) > tolerancja]

liczba_niezgodnosci = len(niezgodnosci_df)

print(liczba_niezgodnosci)

373


### Zad 5

In [17]:
sprzedaz_df = pd.read_csv("raport_kwartalny_sprzedaz.csv", encoding="cp1250")

sprzedaz_df['Wartosc_Obliczona'] = sprzedaz_df['Ilosc_Sprzedana'] * sprzedaz_df['Cena_Jednostkowa_Transakcji']

tolerancja = 0.01

niezgodnosci_df = sprzedaz_df[abs(sprzedaz_df['Wartosc_Sprzedazy'] - sprzedaz_df['Wartosc_Obliczona']) > tolerancja]

if niezgodnosci_df.empty:
    print("Nie wykryto niezgodności w danych.")
else:
    print("Przykładowe wiersze z niezgodnościami:")
    print(niezgodnosci_df[['Wartosc_Sprzedazy', 'Wartosc_Obliczona', 'Ilosc_Sprzedana', 'Cena_Jednostkowa_Transakcji']].head())
    print(f"\nŁączna liczba niezgodności: {len(niezgodnosci_df)}")

Przykładowe wiersze z niezgodnościami:
     Wartosc_Sprzedazy  Wartosc_Obliczona  Ilosc_Sprzedana  \
16             4915.53            4919.60                1   
67              345.93             343.88                4   
73              154.90             150.37                1   
120             713.31             711.04                4   
130            3366.47            3362.82                1   

     Cena_Jednostkowa_Transakcji  
16                       4919.60  
67                         85.97  
73                        150.37  
120                       177.76  
130                      3362.82  

Łączna liczba niezgodności: 373
